In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['creditcard.csv']


In [2]:
cc_data = pd.read_csv("../input/creditcard.csv")

In [3]:
cc_data = cc_data.sample(frac = 1,random_state = 42)

In [4]:
fraud_cc = cc_data.loc[cc_data["Class"]==1]
non_fraud_cc = cc_data.loc[cc_data["Class"]==0][:492]

In [5]:
normal_distributed_data = pd.concat([fraud_cc,non_fraud_cc])

In [6]:
new_cc_data = normal_distributed_data.sample(frac = 1,random_state = 42)

In [7]:
new_cc_data.head(6)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
146884,87941.0,2.054361,-0.122642,-1.245717,0.189567,0.132497,-0.620765,0.059581,-0.148058,0.338940,0.247959,0.504313,0.884524,-0.098371,0.475306,-0.608077,0.237496,-0.702700,-0.199364,0.616409,-0.209856,-0.271204,-0.687048,0.271569,-0.497120,-0.270115,0.208619,-0.076075,-0.075428,0.99,0
95597,65385.0,-2.923827,1.524837,-3.018758,3.289291,-5.755542,2.218276,-0.509995,-3.569444,-1.016592,-4.320536,1.277202,-3.701750,-0.971870,-4.857777,0.090606,-2.801502,-4.186808,-1.648406,1.176446,-0.447039,-0.511657,-0.122724,-4.288639,0.563797,-0.949451,-0.204532,1.510206,-0.324706,1354.25,1
96768,65927.0,0.955253,-0.833374,0.847147,0.725970,-1.322868,-0.238842,-0.565347,0.022443,-0.762247,0.738315,-0.223308,-0.136965,-0.030759,0.119714,1.356862,-1.397447,0.065630,1.061450,-1.912073,-0.294785,-0.110554,-0.129397,-0.040892,0.364872,0.204756,-0.304578,0.055908,0.062489,156.00,0
75511,56098.0,-1.229669,1.956099,-0.851198,2.796987,-1.913977,-0.044934,-1.340739,-0.555548,-1.184468,-3.245109,2.338323,-2.820042,0.763326,-5.419590,0.041433,-4.180758,-4.285071,-2.536452,0.930239,0.270459,1.208054,0.277612,0.019266,0.508529,-0.201183,-0.249600,0.562239,0.075309,170.92,1
255556,157284.0,-0.242245,4.147186,-5.672349,6.493741,1.591168,-1.602523,-0.950463,0.722903,-4.128505,-3.963224,4.469467,-4.655071,-0.441776,-10.149813,0.612798,-0.288725,1.404507,2.132081,0.707200,0.562030,0.249023,-0.480286,-0.286080,-1.153575,-0.035571,0.559628,0.409446,0.221048,0.77,1
208257,137041.0,-0.719251,-0.918495,2.175468,-2.667620,-1.094123,0.355401,-0.581350,0.189298,-1.867227,0.392627,-1.218951,-1.092424,0.886192,-1.032183,0.079028,-0.052301,0.201888,0.451765,-0.441379,0.017703,-0.069422,-0.021690,-0.003213,0.568798,0.320581,-0.159864,0.050443,0.086122,101.00,0


In [8]:
new_cc_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 984 entries, 146884 to 149869
Data columns (total 31 columns):
Time      984 non-null float64
V1        984 non-null float64
V2        984 non-null float64
V3        984 non-null float64
V4        984 non-null float64
V5        984 non-null float64
V6        984 non-null float64
V7        984 non-null float64
V8        984 non-null float64
V9        984 non-null float64
V10       984 non-null float64
V11       984 non-null float64
V12       984 non-null float64
V13       984 non-null float64
V14       984 non-null float64
V15       984 non-null float64
V16       984 non-null float64
V17       984 non-null float64
V18       984 non-null float64
V19       984 non-null float64
V20       984 non-null float64
V21       984 non-null float64
V22       984 non-null float64
V23       984 non-null float64
V24       984 non-null float64
V25       984 non-null float64
V26       984 non-null float64
V27       984 non-null float64
V28       984 non-n

In [9]:
X = new_cc_data.drop("Class",axis = 1)
# X

In [10]:
y = new_cc_data["Class"]
#y

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,stratify = y,random_state = 42)

In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
sc = StandardScaler()
sc.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [15]:
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [16]:
print(X_train_std.shape)
print(X_test_std.shape)

(787, 30)
(197, 30)


# logistic regression

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix

In [18]:
log_model = LogisticRegression(solver = 'lbfgs',random_state = 42)
log_scores = cross_val_score(log_model, X_train_std, y_train, cv = 10)
print("log_scores: ", log_scores)
print("Mean score: ", log_scores.mean())
print("Std deviation: ", log_scores.std())

log_scores:  [0.9125     0.975      0.925      0.96202532 0.93589744 0.96153846
 0.94871795 0.98717949 0.91025641 0.96153846]
Mean score:  0.9479653521583901
Std deviation:  0.024875943538302702


In [19]:
log_model.fit(X_train,y_train)
log_model_score = log_model.score(X_test_std,y_test)
log_model_score

0.883248730964467

In [20]:
log_yy_pred = log_model.predict(X_test_std)

In [21]:
confMatrix = confusion_matrix(y_true=y_test, y_pred=log_yy_pred)
print(confMatrix)

[[99  0]
 [23 75]]


In [22]:
print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=log_yy_pred))
print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=log_yy_pred))
print('F1: %.3f' % f1_score(y_true=y_test, y_pred=log_yy_pred))

Precision: 1.000
Recall: 0.765
F1: 0.867


# SVM

# linear kernel 

In [23]:
from sklearn.svm import SVC

In [24]:
svm_linear_model = SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=42,
  shrinking=True, tol=0.001, verbose=False)

In [25]:
svm_linear_scores = cross_val_score(svm_linear_model, X_train_std, y_train, cv = 10)
print("svm_linear_scores: ", svm_linear_scores)
print("Mean score: ", svm_linear_scores.mean())
print("Std deviation: ", svm_linear_scores.std())

svm_linear_scores:  [0.9125     0.975      0.9375     0.93670886 0.93589744 0.96153846
 0.94871795 0.98717949 0.8974359  0.96153846]
Mean score:  0.9454016553067186
Std deviation:  0.026009236719929392


In [26]:
svm_linear_model.fit(X_train_std,y_train)
svm_linear_score = svm_linear_model.score(X_test_std,y_test)
svm_linear_score

0.9137055837563451

In [27]:
svm_linear_yy_pred = svm_linear_model.predict(X_test_std)

In [28]:
svm_linear_model_confMatrix = confusion_matrix(y_true=y_test, y_pred=svm_linear_yy_pred)
print(svm_linear_model_confMatrix)

[[97  2]
 [15 83]]


In [29]:
print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=svm_linear_yy_pred))
print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=svm_linear_yy_pred))
print('F1: %.3f' % f1_score(y_true=y_test, y_pred=svm_linear_yy_pred))

Precision: 0.976
Recall: 0.847
F1: 0.907


# rbf kernel

In [30]:
svm_rbf_model = SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.9999999999999999,
  kernel='rbf', max_iter=-1, probability=False, random_state=1,
  shrinking=True, tol=0.001, verbose=False)

In [31]:
svm_rbf_scores = cross_val_score(svm_rbf_model, X_train_std, y_train, cv = 10)
print("svm_linear_scores: ", svm_rbf_scores)
print("Mean score: ", svm_rbf_scores.mean())
print("Std deviation: ", svm_rbf_scores.std())

svm_linear_scores:  [0.7625     0.7875     0.825      0.84810127 0.79487179 0.82051282
 0.79487179 0.78205128 0.78205128 0.84615385]
Mean score:  0.8043614086335605
Std deviation:  0.027493410426734057


In [32]:
svm_rbf_model.fit(X_train_std, y_train)
svm_rbf_score = svm_rbf_model.score(X_test_std, y_test)
svm_rbf_score

0.7918781725888325

In [33]:
svm_rbf_yy_pred = svm_rbf_model.predict(X_test_std) 

In [34]:
svm_rbf_model_confMatrix = confusion_matrix(y_true=y_test, y_pred=svm_rbf_yy_pred)
print(svm_rbf_model_confMatrix)

[[64 35]
 [ 6 92]]


In [35]:
print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=svm_rbf_yy_pred))
print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=svm_rbf_yy_pred))
print('F1: %.3f' % f1_score(y_true=y_test, y_pred=svm_rbf_yy_pred))

Precision: 0.724
Recall: 0.939
F1: 0.818


# random forest classification

In [36]:
from sklearn.ensemble import RandomForestClassifier

In [37]:
rand_forest_clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=5,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=12, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [38]:
rand_forest_clf_scores = cross_val_score(rand_forest_clf,X_train_std,y_train,cv = 10)
print("rand_forest_clf_scores:",rand_forest_clf_scores)
print("rand_forest_clf _mean:",rand_forest_clf_scores.mean())
print("rand_forest_clf_std:",rand_forest_clf_scores.std())

rand_forest_clf_scores: [0.925      0.9375     0.925      0.96202532 0.93589744 0.94871795
 0.94871795 0.93589744 0.92307692 0.94871795]
rand_forest_clf _mean: 0.9390550957481338
rand_forest_clf_std: 0.0121675659482233


In [40]:
rand_forest_clf.fit(X_train_std,y_train)
rand_forest_clf_score = rand_forest_clf.score(X_test_std,y_test)
rand_forest_clf_score

0.9035532994923858

In [41]:
rand_for_yy_pred = rand_forest_clf.predict(X_test)

In [42]:
confMatrix = confusion_matrix(y_true = y_test,y_pred = rand_for_yy_pred)
confMatrix

array([[ 5, 94],
       [ 0, 98]])